## Integrating MCP Tool Calls Using MCP Server

In [6]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import init_chat_model

# Load your OPENAI API key from the .env file
load_dotenv(find_dotenv())

# Initialize the chat model with the specified model name
llm = init_chat_model("openai:gpt-4o")

In [ ]:
from typing import Annotated, Optional, List
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_mcp_adapters.client import MultiServerMCPClient

from pydantic import BaseModel, Field

async def create_mcp_graph():
    # Connect to MCP servers
    client = MultiServerMCPClient({
        "math": {
            "command": "python",
            "args": ["calculator_server.py"], 
            "transport": "stdio"
        }
    })
    
    # Discover tools from MCP server
    mcp_tools = await client.get_tools()
    print(f"Found MCP tools: {[tool.name for tool in mcp_tools]}")
    
    # Build the graph with MCP tools
    llm_with_tools = llm.bind_tools(mcp_tools)
    
    class State(TypedDict):
        messages: Annotated[list, add_messages]
        structured_output: Optional[str]
    
    async def chatbot(state: State):  # ← Must be async for MCP
        return {"messages": [await llm_with_tools.ainvoke(state["messages"])]}
    
    
    async def format_result(state: State):
        """Format the conversation into structured output"""
        
        class MathResult(BaseModel):
            calculation: str = Field(description="The mathematical expression that was calculated")
            result: float = Field(description="The result of the calculation")
            steps: List[str] = Field(description="Step by step breakdown of the calculation")
            confidence: str = Field(description="Confidence level of the result")
        
        # Create a summary of the conversation
        conversation_parts = []
        for msg in state["messages"]:  # Don't reverse, keep chronological order
            msg_type = getattr(msg, 'type', 'unknown')
        
            if hasattr(msg, 'content') and msg.content:
                conversation_parts.append(f"{msg_type}: {msg.content}")
        
            # Include tool calls information
            if hasattr(msg, 'tool_calls') and msg.tool_calls:
                for call in msg.tool_calls:
                    conversation_parts.append(f"{msg_type}_tool_call: {call['name']}({call['args']})")
        
            # Include tool results
            if hasattr(msg, 'tool_call_id'):
                tool_name = getattr(msg, 'name', 'unknown_tool')
                conversation_parts.append(f"tool_result_{tool_name}: {msg.content}")
    
        conversation_summary = "\n".join(conversation_parts)
        
        structured_llm = llm.with_structured_output(MathResult)
        
        structured_output = await structured_llm.ainvoke([{
            "role": "user",
            "content": f"extract structured math result from this conversation: {conversation_summary}"
            }])
        
        json_output = structured_output.model_dump() # convert to json
        
        import json
        return {"structured_output": json.dumps(json_output, indent=2)}

    

    graph_builder = StateGraph(State)
    graph_builder.add_node("chatbot", chatbot)
    
    tool_node = ToolNode(tools=mcp_tools)
    graph_builder.add_node("tools", tool_node)
    
    graph_builder.add_node("format_result", format_result)

    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
        {"tools": "tools", END: "format_result"}
    )

    graph_builder.add_edge(START, "chatbot")
    graph_builder.add_edge("tools", "chatbot")
    graph_builder.add_edge("format_result", END)
    
    return graph_builder.compile()


# Cleanup function to close MCP connections
async def cleanup():
    """Simple cleanup when done with demo"""
    try:
        if 'graph' in globals():
            # Close any MCP connections if they exist
            print("Demo cleanup complete")
    except Exception as e:
        print(f"Cleanup error: {e}")

# Call when you're done:
# await cleanup()


# CREATE the graph first
graph = await create_mcp_graph()



Found MCP tools: ['square_number', 'add_numbers', 'subtract_numbers', 'multiply_numbers', 'divide_numbers', 'power', 'square_root']


In [9]:
async def calculate_with_validation(query: str):
    """Calculate with input validation and better error handling"""

    # Check if the query is meaningful
    if not query or len(query.strip()) < 3:
        return "Please provide a valid math question."
    
    try:
        print("Asking a math question...")
        result = await graph.ainvoke({
            "messages": [
                {
                "role": "system",
                "content": "You are a helpful math assistant that can perform complex calculations using external tools.\n"
                "           Use the provided tools one at a time to solve the problem.\n"
                "           Never use the tools in parallel, always wait for the result of one tool before using another.\n"
                "           Follow order of operations strictly.\n",
                },
                {
                    "role": "user", 
                    "content": query + "\n",
                }
            ]
        })

        # Return the result from the LLM
        return result
    except Exception as e:
        return f"Calculation error: {str(e)}"


In [10]:
# Example usage
result = await calculate_with_validation("what's the square root of 156.789, plus 47.234, then multiply by 3?")

Asking a math question...


In [11]:
# Viewing the answer
# Output is an AIMessage
# Accessing the last message content for the final answer
print("Final answer:", result["messages"][-1].content)

Final answer: The result of the calculation is approximately \(179.27\).


In [12]:
# Viewing the structured output
print("Structured output:", result["structured_output"])

Structured output: {
  "calculation": "((sqrt(156.789) + 47.234) * 3)",
  "result": 179.2666243159705,
  "steps": [
    "Calculate the square root of 156.789 to get 12.521541438656824.",
    "Add the result to 47.234 to get 59.75554143865683.",
    "Multiply the result by 3 to get 179.2666243159705."
  ],
  "confidence": "High"
}


In [9]:
await cleanup()  # Call cleanup when done

🧹 Demo cleanup complete
